# AI Agents Crash Course - Part 3 Implementation

This notebook demonstrates the key concepts from the Daily Dose of Data Science AI Agents article by Avi Chawla and Akshay Pachaar.

## What you'll learn:
1. **CrewAI Flows**, a powerful feature that enables the creation of structured, event-driven workflows for AI agents. This article focuses on moving beyond basic agent configurations to building complex, production-ready systems that seamlessly integrate deterministic processes with AI-driven autonomy through flow control mechanisms.

## Prerequisites:
- Install required packages
- Set up API keys
- Choose your LLM provider

## 📦 Installation and Setup

First, let's install the required packages and set up our environment.

In [1]:
# Install required packages (run this cell first)
#!pip install crewai crewai-tools python-dotenv pyyaml IPython

In [ ]:
import os
import yaml
from dotenv import load_dotenv
from IPython.display import Markdown, display
import warnings
warnings.filterwarnings('ignore')

# CrewAI imports
from crewai import Agent, Task, Crew, Process, LLM
from crewai_tools import SerperDevTool, FileReadTool

print("✅ All packages imported successfully!")

## 🔑 Environment Configuration

Set up your environment variables. You have two options:

### Option 1: Create a `.env` file with:
```
SERPER_API_KEY="your-serper-api-key"
AZURE_OPENAI_API_KEY="your-azure-openai-key"
AZURE_OPENAI_ENDPOINT="your-azure-openai-endpoint"
AZURE_OPENAI_API_VERSION="your-azure-openai-api-version"
AZURE_OPENAI_MODEL_NAME="your-azure-openai-model-name"
```

### Option 2: Set them directly in this notebook (not recommended for production):

In [ ]:
import os

# Load environment variables
load_dotenv()

# Option 2: Uncomment and set your API keys directly (not recommended for production)
# os.environ['OPENAI_API_KEY'] = 'your-openai-api-key-here'
# os.environ['SERPER_API_KEY'] = 'your-serper-api-key-here'

# Configure LLM - Choose one of the options below:

# Option A: Local Ollama (as mentioned in the article)
#llm = LLM(
#    model="ollama/llama3.2:1b",
#    base_url="http://localhost:11434"
#)

# Option B: OpenAI GPT-4 (uncomment to use)
# llm = LLM(model="gpt-4")

# Option C: OpenAI GPT-3.5-turbo (cheaper alternative)
# llm = LLM(model="gpt-3.5-turbo")

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## 🤖 Example 1: Basic Flow

In this walkthrough, we’ll explore how CrewAI Flows allow you to effortlessly manage sequences of tasks powered by AI.

To keep things practical, we'll build a simple yet interesting scenario: first generating a random movie genre, then using that genre to suggest a popular movie recommendation.

In [3]:
import openai
from crewai.flow.flow import Flow, start, listen

openai_client = openai.AzureOpenAI(
    api_key=openai_api_key,
    api_version=openai_api_version,
    azure_endpoint=openai_endpoint
)

class MovieRecommendationFlow(Flow):

    @start()
    def generate_genre(self):
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": "Give me a random movie genre.",
                },
            ],
        )

        random_genre = response.choices[0].message.content.strip()

        return random_genre
    

    @listen(generate_genre)
    def recommend_movie(self, random_genre):
    
        response = openai_client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "user",
                    "content": f"Recommend a movie in {random_genre} genre.",
                },
            ],
        )

        movie_recommendation = response.choices[0].message.content.strip()
        
        return movie_recommendation

In [ ]:
flow = MovieRecommendationFlow()
final_result = await flow.kickoff_async()

print(f"\nMovie Recommendation: {final_result}")


While this is a basic demo that could have been done with a simple LLM call, the example was used to explain that this particular design pattern is incredibly powerful for several reasons:

Flows automate AI-driven sequential task execution, removing manual intervention.
Tasks can easily share data, ensuring consistent context management.
You can effortlessly add more tasks and dependencies.